In [6]:
from functools import partial
from typing import Callable

import altair as alt
import numpy as np
import pandas as pd
from code_ramblings.distributions import PoissonNegBinomMixture, compute_value_frequencies


# Altair stuff
alt.data_transformers.enable("vegafusion")
title: Callable[[str], alt.Title] = partial(alt.Title, color="darkslategrey", fontSize=25)
subtitle: Callable[[str], alt.Title] = partial(alt.Title, color="darkgrey", fontSize=16)


In [7]:
SEED = 0
N = 10000

rng = np.random.default_rng(SEED)

poisson = rng.poisson(4, N)
nb = rng.negative_binomial(1, 0.5, N)
nb_poisson = np.concatenate([poisson, nb])

distributions = {
    "Poisson": poisson,
    "Negative Binomial": nb,
    "Mixed": nb_poisson,
}

source = pd.concat([pd.DataFrame(data, columns=["values"]).assign(distribution=dist) for dist, data in distributions.items()])

In [10]:
base = (
    alt.Chart(source)
    .transform_filter(alt.datum.distribution == "Mixed")
    .encode(x=alt.X("values:Q", title="Count"))
    .properties(width=400, height=200)
)

line = base.mark_line().encode(y=alt.Y("count()", title="Count of Distribution")).properties(title=subtitle("Distribution"))
boxplot = base.mark_boxplot().encode(y=alt.Y("distribution", title="Distribution")).properties(title=subtitle("Boxplot"))

alt.vconcat(line, boxplot).properties(title=title('The "scare" real life data'))

alt.VConcatChart(...)

In [11]:
source = pd.concat([pd.DataFrame(data, columns=["values"]).assign(distribution=dist) for dist, data in distributions.items()])

base = (
    alt.Chart(source)
    .mark_bar()
    .encode(
        x=alt.X("values:O", title="Value"),
        y=alt.Y("count()", title="Frequency"),
        color=alt.Color("distribution:N", title="Distribution", sort=["Mixed", "Poisson"]),
    )
)

mixed = base.transform_filter(alt.datum.distribution == "Mixed").properties(title=subtitle('The original "scare" data'))
split = base.transform_filter(alt.datum.distribution != "Mixed").properties(title=subtitle("Splitting the stack"))
offset = split.encode(xOffset=alt.XOffset("distribution:N")).properties(title=subtitle("Two easily modeled distributions side by side"))

alt.vconcat(alt.hconcat(mixed, split), offset).properties(title=title('The "scare" real life data against proper statistics'))

alt.VConcatChart(...)

In [5]:
from pprint import pprint

mixture_model = PoissonNegBinomMixture()
mixture_results = mixture_model.fit(nb_poisson)


distributions = {
    "Mixed": nb_poisson,
    "Model": mixture_model.sample(N * 2, random_state=0),
}

df = compute_value_frequencies(distributions)


comparison = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("value:O", title="Value"),
        y=alt.Y("count:Q", title="Frequency"),
        color=alt.Color("distribution:N", title="Distribution"),
        xOffset=alt.XOffset("distribution:N"),
    )
)

pprint(mixture_results)
display(mixture_model.sample(100, random_state=0))
display(comparison)

MixtureResults(params=MixtureParams(pi=np.float64(0.5040815986738955),
                                    poisson_lambda=np.float64(4.020087314182715),
                                    negbinom_r=np.float64(0.9953834799503147),
                                    negbinom_p=np.float64(0.5061942728959601)),
               log_likelihood=np.float64(-40806.92373863531),
               converged=True,
               n_iterations=373)


array([ 0,  2,  2,  1,  3,  0,  0,  0,  0,  3,  0,  3,  1,  6,  0,  8,  1,
        1,  2,  6,  4,  3,  4,  0,  1,  6,  3,  0,  0,  0,  1,  2,  7,  5,
        0,  4,  5,  0,  3,  5,  0,  3,  0,  4,  1,  2,  5,  3,  6,  0,  1,
        5,  1,  1,  5,  8,  3,  1,  4, 11,  3,  5,  6,  2,  5,  0,  3,  3,
        3,  2,  0,  0,  5,  2,  6,  3,  0,  0,  1,  7,  0,  5,  3,  0,  5,
        1,  0,  0,  3,  0,  0,  0,  7,  0,  0,  1,  3,  0,  1,  0])

alt.Chart(...)